
# Bipartite ER and Analysis


In [1]:
import math
import os
import numpy as np
import pandas as pd
# for june: if you install this u need a newer version for the girth (NetworkX >= 3.2)
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms.bipartite.generators import random_graph as bipartite_random_graph
from networkx.linalg.algebraicconnectivity import algebraic_connectivity, fiedler_vector
from networkx.algorithms.cycles import girth as nx_girth 


Sources:
https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.generators.random_graph.html

In [2]:
# Configuration
n_qubits = 9

# Numbers of checks
checks_list = [4]

# Probabilities
p_list = [0.5, 0.7, 0.85]

# Repeats per (n_checks, p)
repeats = 20

# Random seed base (each repeat uses seed_base + repeat_id)
seed_base = 1234

# Output directory
OUT_DIR = "sweep_outputs"
os.makedirs(OUT_DIR, exist_ok=True)
print("OUT_DIR:", OUT_DIR)


OUT_DIR: sweep_outputs


In [3]:
def make_bipartite_ER(n_qubits: int, n_checks: int, p: float, seed: int | None = None):
    G = bipartite_random_graph(n_qubits, n_checks, p, seed=seed)
    left = sorted([n for n, d in G.nodes(data=True) if d.get("bipartite", -1) == 0])
    right = sorted([n for n, d in G.nodes(data=True) if d.get("bipartite", -1) == 1])
    Q = [f"q{i}" for i in range(n_qubits)]
    C = [f"c{j}" for j in range(n_checks)]
    mapping = {left[i]: Q[i] for i in range(len(left))}
    mapping.update({right[j]: C[j] for j in range(len(right))})
    G = nx.relabel_nodes(G, mapping, copy=True)
    return G, Q, C

def biadjacency_matrix(G: nx.Graph, Q, C):
    B = np.zeros((len(Q), len(C)), dtype=int)
    for u, v in G.edges():
        if u in Q and v in C:
            B[Q.index(u), C.index(v)] = 1
        elif v in Q and u in C:
            B[Q.index(v), C.index(u)] = 1
    return B


In [4]:
# Run sweep
rows = []
run_id = 0

for n_checks in checks_list:
    p_grid = list(p_list)

    for p in p_grid:
        for r in range(repeats):
            run_id += 1
            seed = seed_base + r
            tag = f"checks{n_checks}_p{p:.3f}_r{r:03d}"
            out_dir = os.path.join(OUT_DIR, tag)
            os.makedirs(out_dir, exist_ok=True)

            # generate
            G, Q, C = make_bipartite_ER(n_qubits, n_checks, p, seed=seed)

            # metircs
            degQ = [G.degree(q) for q in Q]
            degC = [G.degree(c) for c in C]
            avg_degQ = float(np.mean(degQ)) if len(degQ) else 0.0
            avg_degC = float(np.mean(degC)) if len(degC) else 0.0
            min_degQ, max_degQ = (int(np.min(degQ)), int(np.max(degQ))) if len(degQ) else (0,0)
            min_degC, max_degC = (int(np.min(degC)), int(np.max(degC))) if len(degC) else (0,0)

            # connectivity
            comps = list(nx.connected_components(G))
            num_components = len(comps)
            is_conn = (num_components == 1)

            # Algebraic connectivity (Fiedler value)
            try:
                lam2 = float(algebraic_connectivity(G))
            except Exception as e:
                lam2 = float("nan")

            # fiedler vector norm 
            try:
                fvec = np.asarray(fiedler_vector(G))
                fvec_norm = float(np.linalg.norm(fvec))
            except Exception:
                fvec_norm = float("nan")

            # girth
            g = nx_girth(G)
            if g == math.inf:
                g_out = float("inf")
            else:
                g_out = int(g)

            # edge stats
            E = G.number_of_edges()
            # bipartite density relative to |Q|*|C|
            density = E / (len(Q) * len(C)) if (len(Q) and len(C)) else 0.0

            # logical qubits (for your note)
            logical = n_qubits - n_checks

            # save artifacts
            # biadjacency
            B = biadjacency_matrix(G, Q, C)
            pd.DataFrame(B, index=Q, columns=C).to_csv(os.path.join(out_dir, "biadjacency.csv"))
            # edge list
            nx.write_edgelist(G, os.path.join(out_dir, "edges.csv"), data=False)

            # degrees table
            deg_df = pd.DataFrame({
                "node": Q + C,
                "partition": ["Q"]*len(Q) + ["C"]*len(C),
                "degree": degQ + degC,
            })
            deg_df.to_csv(os.path.join(out_dir, "degrees.csv"), index=False)

            # metrics row
            rows.append({
                "run_id": run_id,
                "n_qubits": n_qubits,
                "n_checks": n_checks,
                "p": p,
                "repeat": r,
                "edges": E,
                "density_QxC": density,
                "avg_degQ": avg_degQ,
                "min_degQ": min_degQ,
                "max_degQ": max_degQ,
                "avg_degC": avg_degC,
                "min_degC": min_degC,
                "max_degC": max_degC,
                "is_connected": bool(is_conn),
                "num_components": int(num_components),
                "fiedler_lambda2": lam2,
                "fiedler_vec_norm": fvec_norm,
                "girth": g_out,
                "logical_qubits": logical,
                "out_dir": out_dir
            })

In [5]:
# aggregate metrics
metrics_path = os.path.join(OUT_DIR, "metrics.csv")
metrics_df = pd.DataFrame(rows)
metrics_df.to_csv(metrics_path, index=False)
metrics_path, len(metrics_df)

('sweep_outputs\\metrics.csv', 60)